In [1]:
%pip install opencv-python
%pip install ipywidgets
%pip install joblib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import tensorflow as tf
import os
from IPython.display import display
import PIL.Image
import ipywidgets as widgets
from threading import Thread
import cv2
import joblib

2023-11-29 13:36:50.144398: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 13:36:51.377862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
CWD = os.getcwd()
model_path = os.path.join(CWD, ".." ,"models", "yolov8n_float32.tflite")
model_path = os.path.abspath(model_path)
print("model_path:" , model_path)


model_path: /home/shivansh/Desktop/startup/t2d2/models/yolov8n_float32.tflite


In [4]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
# Create a widget for displaying the camera feed
image_widget = widgets.Image(format='jpeg', width=300, height=300)

In [ ]:
# Function to update the image widget with the processed frame
def update_image_widget(frame):
    _, buffer = cv2.imencode('.jpeg', cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    image_widget.value = buffer.tobytes()

In [ ]:

def write_detection_result(detection):
    file_path = os.path.join(CWD, ".." ,"logs", "detection.txt")
    file_path = os.path.abspath(file_path)
    # with open(file_path, "a") as f:
    #     f.write(str(detection) + "\n")
    #     f.close()
    
    
    # create pickle file
    with open(file_path, "wb") as f:
        joblib.dump(detection, f)
    
    return

def save_frame(frame):
    file_path = os.path.join(CWD, ".." ,"logs", "frame.jpg")
    file_path = os.path.abspath(file_path)
    cv2.imwrite(file_path, frame)
    return


# Function to perform object detection and update the image widget
def detect_objects():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
            
        #detect object from image
        
        # frame = cv2.imread("temp.jpg")   
        
        # Object detection
        input_data = preprocess_input(frame)
        print("input_data:", input_data.shape)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        print("output_data", output_data.shape)
        

        process_output(frame, output_data)
        update_image_widget(frame)
        
        save_frame(frame)
        

# Function to preprocess input frame for the TFLite model
def preprocess_input(frame):
    input_shape = input_details[0]['shape']
    input_data = cv2.resize(frame, (input_shape[1], input_shape[2]))
    input_data = np.expand_dims(input_data, axis=0)
    input_data = input_data.astype(np.float32)
    return input_data

# Function to process the output of the TFLite model
def process_output(frame, output_data):
          
    # save detection result without trunkating the array
    write_detection_result(output_data[0])
        
    for detection in output_data[0]:
        print(detection.shape)
        class_probs = detection[5:]  # Adjust the index based on your model
        class_id = np.argmax(class_probs)
        score = class_probs[class_id]
            

        if score > 0.5:  # Adjust the threshold as needed
            box = detection[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            label = f"Class: {class_id}, Score: {score:.2f}"
            cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [ ]:
# # Start the object detection thread
# object_detection_thread = Thread(target=detect_objects)
# object_detection_thread.start()

# # Display the image widget
# display(image_widget)
detect_objects()

In [ ]:
import joblib
with open("/home/shivansh/Desktop/startup/t2d2/logs/detection.txt", "rb") as f:
    detection = joblib.load(f)
    print("detection:", detection)

In [ ]:
detection.__class__.__name__

detection.shape

In [ ]:
#  print detection without truncation
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

detection

In [ ]:
# reshape to 1D array
detection = detection.reshape(-1)

In [ ]:
%pip install matplotlib

In [ ]:
# scatter log plot
import matplotlib.pyplot as plt
plt.scatter(range(len(detection)), detection)
